In [ ]:
'''
XGboost stacking
'''

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split
import os
from tqdm import tqdm
import seaborn as sns
import xgboost as xgb
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score,classification_report

In [ ]:
train_df = pd.read_csv('../input/pandaprobatpu1536768models/train_tpu1536_prediction.csv')
print(train_df.shape)
train_df.head()

In [ ]:
sns.countplot(train_df['label'])

In [ ]:
train_df['label'].value_counts()

In [ ]:
train_df_not_01 = train_df[(train_df['label'] != 0) & (train_df['label'] != 1)]
train_class0 = train_df[train_df['label'] == 0].sample(frac=0.9999)
train_class1 = train_df[train_df['label'] == 1].sample(frac=0.9999)

print(train_df_not_01['label'].value_counts())
print(train_class0['label'].value_counts())
print(train_class1['label'].value_counts())

In [ ]:
final_train = train_df_not_01
final_train = final_train.append(train_class0)
final_train = final_train.append(train_class1)
final_train.reset_index(inplace = True) 

print(final_train['label'].value_counts())
#print(final_train.head())

xtrain, xval, ytrain, yval = train_test_split(new_train_df["image_path"], new_train_df[["col0","col1","col2","col3","col4"]], test_size = 0.15, stratify = new_train_df[['label']])

df_train = pd.DataFrame({"image_path":xtrain, "col0":ytrain['col0'], "col1":ytrain['col1'], "col2":ytrain['col2'], "col3":ytrain['col3'], "col4":ytrain['col4'] })
df_val = pd.DataFrame({"image_path":xval, "col0":yval['col0'], "col1":yval['col1'], "col2":yval['col2'], "col3":yval['col3'], "col4":yval['col4'] })


df = pd.read_csv(LABELS).set_index('image_id')
files = sorted(set([p[:32] for p in os.listdir(TRAIN)]))
df = df.loc[files]
df = df.reset_index()

### CV ,  Fold cretion

In [ ]:
from sklearn.model_selection import StratifiedKFold

df = final_train
nfolds = 10

splits = StratifiedKFold(n_splits=nfolds, random_state=2020, shuffle=True)
splits = list(splits.split(df,df.label))

folds_splits = np.zeros(len(df)).astype(np.int)
for i in range(nfolds): folds_splits[splits[i][1]] = i

df['split'] = folds_splits
df.head()

In [ ]:
df['split'].value_counts()

In [ ]:
def run_train():
    for fold_number in range(nfolds):
        print('Training started for Fold :' + str(fold_number))
    
        train_df = df[(df.split != fold_number)]
        valid_df = df[(df.split == fold_number)]
        
        #print(train_df.shape + valid_df.shape )
    
        train_model(train_df, valid_df, fold_number)

    df1 = train_df.iloc[:,1:7]
    df2 = train_df.iloc[:,7:13]
    df3 = train_df.iloc[:,13:19]
    df4 = train_df.iloc[:,19:25]
    
        #X_train = (df1.to_numpy() + df2.to_numpy() + df3.to_numpy() + df4.to_numpy()) / 4
    #y_train = train_df.label.to_numpy()
    #X_train = train_df.iloc[:,13:19]
    #y_train = train_df.label
    
    xgb_model = xgb.XGBClassifier(n_estimators=100,learning_rate=0.2,objective='multi:softmax',
                                  max_depth=3, eval_metric = 'mlogloss' )
    
    # best score
    #> 130 4 .3
    #> 120 4 .3
    
    # For 5 fold
    #({'eta': 0.2, 'max_depth': 4, 'n_estimators': 100}, 0.7147827829075659)
    # For 20
    #{'eta': 0.1, 'max_depth': 6, 'n_estimators': 180}0.719783081226504)
    xgb_model.fit(X_train, y_train, verbose=True)

    vdf1 = valid_df.iloc[:,1:7]
    vdf2 = valid_df.iloc[:,7:13]
    vdf3 = valid_df.iloc[:,13:19]
    vdf4 = valid_df.iloc[:,19:25]
    
    X_test = (vdf1.to_numpy() + vdf2.to_numpy() + (1.2*vdf3.to_numpy()) + vdf4.to_numpy()) / 4
    y_test = valid_df.label.to_numpy()
    
    X_test = valid_df.iloc[:,13:19]
    y_test = valid_df.label
    predictions = xgb_model.predict(X_test)



In [ ]:
X_train = train_df.iloc[:,0:12]
X_train.head()

In [ ]:
import pickle
def train_model(train_df, valid_df, fold_number):
    
    X_train = train_df.iloc[:,0:12]
    y_train = train_df.label
    
    xgb_model = xgb.XGBClassifier(n_estimators=180,learning_rate=0.1,objective='multi:softmax',
                                  max_depth=6, eval_metric = 'mlogloss' )
  

    xgb_model.fit(X_train, y_train, verbose=True)
    
    X_test = valid_df.iloc[:,0:12]
    y_test = valid_df.label
    predictions = xgb_model.predict(X_test)
    
    #print(predictions)
    #print(cohen_kappa_score(y_test, predictions,weights='quadratic'))
    model_score = cohen_kappa_score(y_test, predictions,weights='quadratic')
    print(model_score)
    model_name = 'xgboost-' + str(fold_number) + '-' + str(model_score) + '.dat'
    #pickle.dump(xgb_model, open(model_name, 'wb'))
    print(classification_report(y_test, predictions))
    print('---------------------------------------------------------------')

In [ ]:
run_train()

# GridSearch

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
     "objective"    : 'multi:softmax',
     "num_classes"  : 6
     #"min_child_weight" : [ 1, 3, 5, 7 ],
     #"gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
     #"colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
     }

parameters = {
     "eta"          : [ 0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
     "max_depth"    : [ 3 , 4, 5, 6],
     "n_estimators" : [50, 60,100,110,120,130,140,150,160,170,180,190,200,210,220]
     #"min_child_weight" : [ 1, 3, 5, 7 ],
     #"gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
     #"colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
     }

clf = xgb.XGBClassifier(**params)

grid = GridSearchCV(clf,
                    parameters, n_jobs=4,
                    scoring="accuracy",
                    cv=10)
X_train = train_df.iloc[:,0:12]
y_train = train_df.label

grid.fit(X_train, y_train, verbose=True)

import sklearn
sorted(sklearn.metrics.SCORERS.keys())

In [ ]:
grid.best_params_, grid.best_score_

In [ ]:
grid.cv_results_